### Chapter 13 : General Linear Models


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats
import statsmodels.api as sm

In [2]:
# Helper functions imported from textbook
def p_rtail(sampdist,tstat): 
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() # get x and y values from plot 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = x > tstat, color='r') 
    plt.fill_between(x,y, where = x <= tstat, color='g') 
    plt.show() 
    pvalue = np.sum(sampdist < tstat) / len(sampdist) 
    return pvalue 

def p_ltail(sampdist,tstat): 
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = x < tstat, color='r') 
    plt.fill_between(x,y, where = x >= tstat, color='g') 
    plt.show() 
    pvalue = np.sum(sampdist > tstat) / len(sampdist) 

    return pvalue 

def p_2tail(sampdist,tstat):
    points = sns.kdeplot(sampdist).get_lines()[0].get_data() 
    hyp = np.mean(sampdist) 
    cutoff1 = hyp - np.abs(hyp - tstat) 
    cutoff2 = hyp + np.abs(hyp - tstat) 
    x = points[0] 
    y = points[1] 
    plt.fill_between(x,y, where = (x < cutoff1) | (x > cutoff2), color='r') 
    plt.fill_between(x,y, where = (x >= cutoff1) & (x <= cutoff2), color='g') 
    plt.show() 
    pvalue = np.sum((sampdist < cutoff1) | (sampdist > cutoff2)) / len(sampdist) 
    return pvalue 

Read the file CA.COVID.csv 

2.Are independent variables statistically significant? What is the marginal effect of the two independent variables? Are they practically significant? What do you conclude? 

3.Compute the AIC and BIC values for both the Poisson regression model and the baseline model. 

4.Does the Poisson regression model add value over the baseline model, based on AIC and BIC values? 

In [3]:
#diabetes csv imported into the directory.
covid = pd.read_csv("CA COVID.csv")
covid.head(5)

,County,FACILITY NAME,COUNTY,FACILITY ID,AVAILABLE BEDS,AVAILABLE BEDS CAPABLE OF ISOLATION,NEW CONFIRMED POSITIVE RESIDENTS,CURRENT ACTIVE CASES RESIDENTS,CUMULATIVE POSITIVE RESIDENTS,COVID-RELATED RESIDENT DEATHS,NEW CONFIRMED POSITIVE HCW,CURRENT ACTIVE HCW,CUMULATIVE POSITIVE HCW,COVID-RELATED HCW DEATHS
0,Alameda,ALAMEDA COUNTY MEDICAL CENTER D/P SNF,ALAMEDA,140000321,7,4,0,0,86,0,0,<11,76,0
1,Alameda,ALAMEDA HEALTHCARE & WELLNESS CENTER,ALAMEDA,20000043,54,8,0,0,88,<11,0,0,55,0
2,Alameda,ALAMEDA HOSPITAL - SOUTHSHORE CONVALESCENT,ALAMEDA,630011864,1,1,0,0,<11,0,0,0,<11,0
3,Alameda,ALAMEDA HOSPITAL D/P SNF,ALAMEDA,140000686,3,1,0,0,<11,0,0,0,13,0
4,Alameda,ALL SAINT'S MAUBERT,ALAMEDA,630013891,2,0,0,0,0,0,0,0,<11,0


In [24]:
size =covid['AVAILABLE BEDS']
patients_I = covid['CUMULATIVE POSITIVE RESIDENTS ']
patients_D = covid['COVID-RELATED RESIDENT DEATHS']
staff_I = covid['CUMULATIVE POSITIVE HCW ']
staff_D = covid['COVID-RELATED HCW DEATHS ']
beds = covid['AVAILABLE BEDS']

df = pd.DataFrame(data = {'size':size, 'patients_I':patients_I, 'staff_I':staff_I, 'patients_D':patients_D, 'staff_D':staff_D, 'beds' : beds})
df.head()

,size,patients_I,staff_I,patients_D,staff_D,beds
0,7,86,76,0,0,7
1,54,88,55,<11,0,54
2,1,<11,<11,0,0,1
3,3,<11,13,0,0,3
4,2,0,<11,0,0,2


1.Using the maximum likelihood approach develop a Poisson regression model for the cumulative number of patient infections based on the cumulative number of staff infections and the number of available beds. 

In [25]:
df.loc[df['patients_I'] == '<11','patients_I'] = 5
df.loc[df['staff_I'] == '<11','staff_I'] = 5

In [26]:
df.head(5)

,size,patients_I,staff_I,patients_D,staff_D,beds
0,7,86,76,0,0,7
1,54,88,55,<11,0,54
2,1,5,5,0,0,1
3,3,5,13,0,0,3
4,2,0,5,0,0,2


In [28]:
print(patients_I.mean())

TypeError: Could not convert 8688<11<110<11<1122<111516720<11152029<112010450113844889812012958286990<1124<1148023282066234837114<111222218023837025377029<115326<11502738920498057<11102691413711696166<11185918010336611121062632149014398331<113010653671350465003366<1118091016<112532<11932577221<1133174401071414053<1105130522648765557322216410253869856431048524230172713057<1139191705318<1159001511852275418<1104847799732153117<116312887152161924611099130125547638664552120769515615715<11<116837127<111689276<111963699133116558299491334422422115762394<11402926925311611296114151271212377710745108401224013805048911381066312421107<11134243361479615612234943836153141288090121147210398765294642108561972661761347027<11854261117966<11160525224267513233112677146473406613623628308062031<1111640131124<115712263777856601271243863884160<1181281091078711632103937711099569910129811410635899391711612213497106<1188586149171075113213800152<1187289510197504150701178256162<1168497311762102751689896648495070362

In [27]:
def LLpois(params):
    b0, b1,b2 = params
    X = b0 + b1*df['staff_I'] + b2*df['beds']
    lam = np.exp(X)
    p = scipy.stats.poisson.pmf(df['patients_I'], lam)
    LL = np.sum(np.log(p))
    return(-1*LL)

res5 = scipy.optimize.minimize(LLpois, x0 = [np.log(5.96), 0, 0], method = 'Nelder-Mead')
print(res5)

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
#mean
print(diabetes.Pregnancies.mean())
print(type(diabetes.Pregnancies))
print(diabetes.shape)

In [ ]:
from scipy.stats import poisson
x= diabetes.Pregnancies.sample(n=200).to_numpy()
#print(x)
lam = 4

#since there is no decimal pregnancies, we will round it up to 4

p = poisson.pmf(k = x, mu = lam) 

L = np.prod(p) 

print(L) 

2. What is the 95% confidence interval for the parameter? 

In [ ]:
np.random.seed(987654321) 

paramdist = [diabetes['Pregnancies'].sample(n=len(diabetes), replace=True).mean() for _ in range(10000)]
sns.kdeplot(paramdist) 

plt.show() 

In [ ]:
q1 = np.percentile(paramdist, 0.05/2 * 100) 

q2 = np.percentile(paramdist, (1-(0.05/2)) * 100) 

sns.kdeplot(paramdist) 

plt.axvline(q1, color='r', linestyle='-') 

plt.axvline(q2, color='r', linestyle='-') 

plt.show() 

print("95% Confidence Interval = [{:.2f}, {:.2f}]".format(q1,q2)) 

3. Assuming that the variable BMI follows a normal distribution, obtain the maximum likelihood estimate of the mean and the standard deviation.  

In [ ]:
x= diabetes.BMI.sample(n=200).to_numpy()
#print(x)
lam = round(diabetes.BMI.std())
print(lam)

#since there is no decimal pregnancies, we will round it up to 4

p = poisson.pmf(k = x, mu = lam) 

L = np.prod(p) 

print(L) 

In [ ]:
diabetes.BMI.std()

In [ ]:
x= diabetes.BMI.sample(n=100).to_numpy()
#print(x)
lam = round(diabetes.BMI.mean())
print(lam)

#since there is no decimal pregnancies, we will round it up to 4

p = poisson.pmf(k = x, mu = lam) 

L = np.prod(p) 

print(L) 

In [ ]:
np.random.seed(987654321) 

paramdist = [diabetes['BMI'].sample(n=len(diabetes), replace=True).mean() for _ in range(10000)]
sns.kdeplot(paramdist) 

plt.show() 

q1 = np.percentile(paramdist, 0.05/2 * 100) 

q2 = np.percentile(paramdist, (1-(0.05/2)) * 100) 

sns.kdeplot(paramdist) 

plt.axvline(q1, color='r', linestyle='-') 

plt.axvline(q2, color='r', linestyle='-') 

plt.show() 

print("95% Confidence Interval = [{:.2f}, {:.2f}]".format(q1,q2)) 

5. Using the maximum likelihood approach estimate the population parameter for the binary variable Outcome.   

In [ ]:
import scipy

def LLbinary(pi): 
    p = np.where(diabetes.Outcome == 1, pi, 1 - pi) 
    LL = np.sum(np.log(p)) 
    return(-1*LL) 

constraint = ({'type': 'eq', 'fun': lambda x: x[0]+(1-x[0])}) 

res1 = scipy.optimize.minimize(LLbinary, x0 = .5, constraints=constraint, method='Nelder-Mead') 

print(res1) 

In [ ]:
class MyProbit(GenericLikelihoodModel): 
    def loglike(self, params):
        exog = self.exog 
        endog = self.endog 
        q = 2 * endog - 1 
        return stats.norm.logcdf(q*np.dot(exog, params)).sum() 

sm_probit_manual = MyProbit(endog, exog, method='lbfgs').fit() 

print(sm_probit_manual.summary()) 